# Sentiment Analysis con KNN
## Clasificador en C++ 💪💪
Vamos a probar a nuestro bichito


Definir los path al ejecutable de python 3.6 y sus librerías,
de acuerdo al virtual env que estén corriendo.

In [1]:
!cd .. && git submodule init
!cd .. && git submodule update
!cd .. && mkdir build
!cd ../build/ && rm -rf *
!cd ../build && cmake \
  -DPYTHON_EXECUTABLE="$(which python)" \
  -DCMAKE_BUILD_TYPE=Release ..
!cd ../build && make install


mkdir: no se puede crear el directorio «build»: El archivo ya existe
-- The C compiler identification is GNU 4.8.4
-- The CXX compiler identification is GNU 4.8.4
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
Release mode
-- Found PythonInterp: /home/alicia/.pyenv/versions/3.6.5/bin/python (found version "3.6.5") 
-- Found PythonLibs: /home/alicia/.pyenv/versions/3.6.5/lib/libpython3.6m.a
-- pybind11 v2.3.dev0
-- Performing Test HAS_FLTO
-- Performing Test HAS_FLTO - Success
-- LTO enabled
CMAKE_INSTALL_PREFIX=/home/alicia/Documentos/Metodos Numericos/TP2/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c
-- Configuring done
-- Generating done
-- Build files have been written to: /home/

In [2]:
# Verifico la correcta instalación. Si no falla el import está OK
!pwd
!python --version
import sentiment
from sentiment import entrenar
from sentiment import generate_vocabulary
from sentiment import entrenar_varios

/home/alicia/Documentos/Metodos Numericos/TP2/metodos-tp2/metnum-tp2-20191c/metnum-tp2-20191c/notebooks
Python 3.6.5


In [34]:
%load_ext autoreload
%autoreload 2
import pandas as pd

!cd ../data && tar -xvf *.tgz

df = pd.read_csv("../data/imdb_small.csv", index_col=0)

print("Cantidad de documentos: {}".format(df.shape[0]))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
./._imdb_small.csv
imdb_small.csv
Cantidad de documentos: 12500


In [35]:
df.describe()

,type,review,label,file
count,12500,12500,12500,12500
unique,2,12469,2,12085
top,test,"This should have been a short film, nothing mo...",neg,10112_1.txt
freq,6275,2,6322,2


In [36]:
text_train = df[df.type == 'train']["review"]
label_train = df[df.type == 'train']["label"]
text_test = df[df.type == 'test']["review"]
label_test = df[df.type == 'test']["label"]

print("Cantidad de instancias de entrenamiento = {}".format(len(text_train)))
print("Cantidad de instancias de test = {}".format(len(text_test)))


Cantidad de instancias de entrenamiento = 6225
Cantidad de instancias de test = 6275


In [37]:
def particion_criticas_equilibradas(text_train, label_train, cant_elems):
    dict_texto = text_train.to_dict()
    dict_label = label_train.to_dict()
    claves_positivos = [x for x in dict_label if dict_label[x] == 'pos']
    claves_negativos = [x for x in dict_label if dict_label[x] == 'neg']
    posiciones_elegidas_pos = random.sample(range(0, len(claves_positivos)), int(cant_elems/2))
    posiciones_elegidas_neg = random.sample(range(0, len(claves_negativos)), int(cant_elems/2))
    nuevo_text_train = {}
    nuevo_label_train = {}
    for i in posiciones_elegidas_pos:
        nuevo_label_train[claves_positivos[i]] = 'pos'
        nuevo_text_train[claves_positivos[i]] = dict_texto[claves_positivos[i]]
    for i in posiciones_elegidas_neg:
        nuevo_label_train[claves_negativos[i]] = 'neg'
        nuevo_text_train[claves_negativos[i]] = dict_texto[claves_negativos[i]]
    return pd.Series(nuevo_text_train), pd.Series(nuevo_label_train)

In [7]:
print("Class balance : {} pos {} neg".format(
    (label_train == 'pos').sum() / label_train.shape[0], 
    (label_train == 'neg').sum() / label_train.shape[0]
))

Class balance : 0.49493975903614457 pos 0.5050602409638554 neg


In [23]:
print(X_test.shape)

(6275, 1565)


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
import random

#vectorizer = CountVectorizer(max_df=0.90, min_df=0.01, max_features=5000)
vectorizer = CountVectorizer(stop_words="english", analyzer='word', max_df=0.3, min_df=0.01, max_features=5000)
voc = vectorizer.fit(text_train)

print("Vocabulary content:\n {}".format(voc.vocabulary_))

criticas_a_experimentar = 1000
text_train, label_train = particion_criticas_equilibradas(text_train,label_train,criticas_a_experimentar)
criticas_a_experimentar = 1000
text_test, label_test = particion_criticas_equilibradas(text_test,label_test,criticas_a_experimentar)


X_train, y_train = vectorizer.transform(text_train), (label_train == 'pos').values
X_test, y_test = vectorizer.transform(text_test), (label_test == 'pos').values




Vocabulary content:
 {'spoiler': 1286, 'read': 1112, 'll': 810, 'know': 755, 'horrible': 666, 'rent': 1141, 'night': 942, 'major': 842, 'results': 1146, 'created': 300, 'waste': 1496, 'building': 182, 'involved': 718, 'leading': 778, 'addition': 39, 'new': 938, 'length': 788, 'necessary': 933, 'different': 364, 'rest': 1144, 'shocking': 1224, 'effects': 417, 'result': 1145, 'right': 1155, 'seriously': 1211, 'probably': 1074, 'worst': 1543, 've': 1466, 'seen': 1201, 'year': 1556, 'low': 835, 'budget': 180, 'acting': 26, 'special': 1282, 'characters': 215, 'clichã': 242, 'act': 24, 'stupid': 1327, 'predictable': 1065, 'ways': 1504, 'walking': 1487, 'dark': 321, 'looking': 819, 'cat': 201, 'falling': 486, 'catch': 202, 'small': 1257, 'room': 1164, 'films': 527, 'takes': 1358, 'possible': 1056, 'cut': 315, 'earlier': 406, 'shots': 1229, 'filming': 524, 'make': 843, 'look': 817, 'reminds': 1140, 'guys': 616, 'working': 1539, 'music': 919, 'children': 226, 'brings': 175, 'aspect': 97, 'compu

In [39]:
print(X_test.shape)

(1000, 1565)


In [10]:
import sentiment
import numpy as np


clf = sentiment.KNNClassifier(715)#100neigbourds

clf.fit(X_train.toarray(), y_train)
#print(X_train)
#print((clf.vectores() != X_train))
#print((clf.labels() != y_train))


#Vemos que son iguales
for index, (first, second) in enumerate(zip(y_train, clf.labels().astype(bool))):
    if first != second:
        print(index, second)
        



In [10]:
%%time
from sklearn.metrics import accuracy_score
import time
start_time = time.clock()
y_pred = clf.predict(X_test.toarray())
print (time.clock() - start_time, "segundos en correr {} criticas".format(len(y_pred)))
acc = accuracy_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))

114.567185 segundos en correr 6275 criticas
Accuracy: 0.7853386454183267
CPU times: user 1min 54s, sys: 520 ms, total: 1min 54s
Wall time: 1min 54s


PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)

[[ 0.12155041  0.00149697  0.00190066 ...  0.00281581  0.00251909
  -0.0007818 ]
 [ 0.00149697  0.11074085  0.00088758 ...  0.00641494  0.00147554
   0.00579612]
 [ 0.00190066  0.00088758  0.10957476 ...  0.00230012  0.00107674
   0.00052297]
 ...
 [ 0.00281581  0.00641494  0.00230012 ...  0.12994766  0.00360788
   0.01734973]
 [ 0.00251909  0.00147554  0.00107674 ...  0.00360788  0.10981296
   0.00230545]
 [-0.0007818   0.00579612  0.00052297 ...  0.01734973  0.00230545
   0.12885488]]
[[ 9.95661652e-03  3.93528124e-05  4.66173618e-05 ...  4.03264902e-04
  -5.89024376e-05  3.32166136e-06]
 [ 3.93528124e-05  2.07206250e-03  6.56978975e-06 ...  3.11308131e-04
   6.27870065e-05  1.28418978e-04]
 [ 4.66173618e-05  6.56978975e-06  2.18500574e-03 ...  1.22078239e-04
   6.80120728e-05  4.35441912e-05]
 ...
 [ 4.03264902e-04  3.11308131e-04  1.22078239e-04 ...  7.97942845e-03
   2.16268891e-04  6.53611672e-04]
 [-5.89024376e-05  6.27870065e-05  6.80120728e-05 ...  2.16268891e-04
   2.73887122

In [9]:
import numpy as np
from IPython.core.debugger import set_trace


class KNNClassifier:
    def __init__(self, n_neighbors, metric='l2'):
        self.n_neighbors = n_neighbors
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
    def _predict_row(self, row):
        A = np.argsort(np.sum((np.square(self.X-row)), axis=1))        
        
        k = np.zeros(self.n_neighbors)
        for i in range(k.shape[0]):
            k[i] = self.y[A[i]]
        
        k = np.bincount(k.astype(int))        
        

        return np.argmax(k)
    
    def predict(self, X, y=None):
        ret = np.zeros(X.shape[0])
        for k, row in enumerate(X):
            ret[k] = self._predict_row(row)
        return ret

In [15]:
import numpy as np

M = (X_train - X_train.mean(axis=0))

cov_matrix = M.T @ M / (M.shape[0]-1) 
w, V = np.linalg.eig(cov_matrix)

# A veces aparecen números complejos acá. Los descartamos
w = np.real(w)
V = np.real(V)

X_pca_train = X_train @ V[:, :30]
X_pca_test = X_test @ V[:, :30]

clf = KNNClassifier(715)
clf.fit(X_pca_train, y_train)

In [16]:
%%time
from sklearn.metrics import accuracy_score
y_pred = clf.predict(X_pca_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {}".format(acc))

Accuracy: 0.5064541832669323
CPU times: user 40.2 s, sys: 4 ms, total: 40.2 s
Wall time: 40.2 s


In [18]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

pca = PCA(30)
pca.fit(X_train.toarray())
X_train_posta = pca.transform(X_train.toarray())
X_test_posta = pca.transform(X_test.toarray())

clf3 = sentiment.KNNClassifier(715) #hago un clf nuevo para no mezclar
clf3.fit(X_train_posta, y_train)
y_pred = clf3.predict(X_test_posta)

acc = accuracy_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))

Accuracy: 0.6956175298804781
CPU times: user 5.6 s, sys: 824 ms, total: 6.43 s
Wall time: 3.96 s


In [10]:
%%time
from sklearn.metrics import accuracy_score

clf2 = sentiment.PCA(10) #10 es la alfa, la cantidad de autovectores/valores que usamos
#Si X_train es de dimension n x m, n  siendo instancias y m cantidad de variables
#V matriz cambio de base va a ser de dimension m x alfa



CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 24.7 ms


In [11]:
%%time
clf2.fit(X_train.toarray(), 1000, 10)#Genera y guarda matriz cambiodeBase de X_train, a partir de la matriz de covarianza.
X_pda_train = clf2.transform(X_train.toarray())#Hace X_train = X_train *V(cambiodeBase generado por el fit)
#La matriz resultante que vamos a usar en el knn tiende dimension n x alfa
#Como cambiamos de base, tenemos que cambiar la base de las cosas en test para que vivan en el mismo espacio
X_pda_test = clf2.transform(X_test.toarray())
#Luego se sigue normalmente


CPU times: user 1min 27s, sys: 908 ms, total: 1min 28s
Wall time: 1min 27s


In [12]:
%%time
clf3 = sentiment.KNNClassifier(715) #hago un clf nuevo para no mezclar
clf3.fit(X_pda_train, y_train)
y_pred = clf3.predict(X_pda_test)

acc = accuracy_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))

Accuracy: 0.6162549800796813
CPU times: user 1.69 s, sys: 20 ms, total: 1.71 s
Wall time: 1.7 s


In [17]:
%%time
from sklearn.metrics import accuracy_score
import time
start_time = time.clock()
vecinos = [1] + [15*x for x in range(1,10)]
with open("positibidad_01.txt", "w") as f:
    for k in vecinos:
        clf = sentiment.KNNClassifier(k)
        clf.fit(X_train.toarray(), y_train)
        y_pred = clf.predict(X_test.toarray())
        for elem in y_pred:
            f.write(" {}".format(int(elem)))
        print(k/3000)
acc = accuracy_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))
print (time.clock() - start_time, "segundos en correr {} criticas".format(len(y_pred)))

0.0003333333333333333
0.005
0.01
0.015
0.02
0.025
0.03
0.035
0.04
0.045
Accuracy: 0.7595219123505976
1025.74343 segundos en correr 6275 criticas
CPU times: user 17min 2s, sys: 3.72 s, total: 17min 5s
Wall time: 17min 1s


In [14]:
%%time

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import time

start_time = time.clock()
alpha = 80
fprogreso = open("progreso_pda_iter2_01.txt", "w")
clf = sentiment.PCA(alpha)
with open("positividad_pda_iter2_01.txt", "w") as f:
    for num in range(100, 2000, 250):
        clf.fit(X_train.toarray(), num, alpha)
        X_pda_train = clf.transform(X_train.toarray())
        X_pda_test = clf.transform(X_test.toarray())
        clf3 = sentiment.KNNClassifier(100) 
        clf3.fit(X_pda_train, y_train)
        y_pred = clf3.predict(X_pda_test)
        for elem in y_pred:
            f.write(" {}".format(int(elem)))
        fprogreso.write(" {}".format(int(num)))
fprogreso.close()    
acc = accuracy_score(y_test, y_pred.astype(bool))
prec = precision_score(y_test, y_pred.astype(bool))
recall = recall_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))
print("Accuracy: {}".format(prec))
print("Accuracy: {}".format(recall))
print (time.clock() - start_time, "segundos en correr {} criticas".format(len(y_pred)))            
       



Accuracy: 0.6231075697211156
Accuracy: 0.575776397515528
Accuracy: 0.897965773329028
2759.7374720000003 segundos en correr 6275 criticas
CPU times: user 45min 47s, sys: 12.4 s, total: 45min 59s
Wall time: 45min 33s


In [42]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import sentiment
import numpy as np
from sklearn.decomposition import PCA
import time
start_time = time.clock
print(type(start_time.type)

AttributeError: 'builtin_function_or_method' object has no attribute 'type'

In [40]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import sentiment
import numpy as np
from sklearn.decomposition import PCA
import time

start_time = time.clock()
alpha = 100
f= open("samples_size100_time_alpha", "w")
for alpha in range (10,100,10 )
    start_time = time.clock()
    num_iteraciones = 500
    clf = sentiment.PCA(alpha)
    clf.fit(X_train.toarray(), num_iteraciones, alpha)
    f.write(" {}".format(time.clock() - start_time))
f.close()       

CPU times: user 3min 52s, sys: 1.81 s, total: 3min 54s
Wall time: 3min 50s


In [25]:
fprogreso = open("progreso_pda_alpha_01.txt", "w")
with open("positividad_pda_alpha_01.txt", "w") as f:
    for n in range(2, 100, 5):        
        clf.fit(X_train.toarray(), num_iteraciones, n)
        X_pda_train = clf.transform(X_train.toarray())
        X_pda_test = clf.transform(X_test.toarray())
        clf3 = sentiment.KNNClassifier(715) 
        clf3.fit(X_pda_train, y_train)
        y_pred = clf3.predict(X_pda_test)                
        for elem in y_pred:            
            f.write(" {}".format(int(elem)))        
        fprogreso.write(" {}".format(int(n)))
    f.close()
fprogreso.close()    
acc = accuracy_score(y_test, y_pred.astype(bool))
prec = precision_score(y_test, y_pred.astype(bool))
recall = recall_score(y_test, y_pred.astype(bool))
print("Accuracy: {}".format(acc))
print("Precision: {}".format(prec))
print("Recall: {}".format(recall))
print (time.clock() - start_time, "segundos en correr {} criticas".format(len(y_pred))) 

Accuracy: 0.6409561752988048
Precision: 0.5842988413903316
Recall: 0.9444623829512432
396.561365 segundos en correr 6275 criticas


In [ ]:
n_components = [x*2 for x in range(1,50)]
with open("positividad_real_no_fake2.txt", "w") as f:
    for n in n_components:             
        for elem in y_test:            
            f.write(" {}".format(int(elem)))        
    f.close()

In [13]:
%%time
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import sentiment
import numpy as np
from sklearn.decomposition import PCA
import time

start_time = time.clock()
alpha = 80
num_iteraciones = 1000
clf = sentiment.PCA(alpha)
clf.fit(X_train.toarray(), num_iteraciones, alpha)

clf.fit(X_train.toarray(), num_iteraciones, 10)




CPU times: user 6min 37s, sys: 3.44 s, total: 6min 40s
Wall time: 6min 34s


In [22]:
clf.fit(X_train.toarray(), num_iteraciones, 20)
X_pdatrain =clf.transform(X_train.toarray())
X_pdatest = clf.transform(X_test.toarray())
print(X_pdatest)
print(X_pdatest.shape)

[[-1.97686781e+00  4.19527018e-01  1.07058507e+00 ... -3.29344247e-01
  -4.96230906e-01 -4.35108291e-02]
 [-2.46798871e+00  5.29457849e-01  5.15272826e-01 ... -1.29535469e+00
   4.69122471e-01 -2.69031824e-01]
 [-2.63618079e-01 -3.37334380e-03 -1.28825474e-01 ...  6.99147296e-02
  -1.06016402e-01 -5.07839531e-02]
 ...
 [-4.41264524e+00 -1.01503753e+00 -1.16829444e+00 ...  1.03958805e+00
  -6.85012169e-01  2.65175893e-02]
 [-1.13784782e+00  2.66447186e-01 -1.09570287e-01 ... -1.35121219e-01
   6.13693386e-02  4.69386744e-01]
 [-3.98783282e+00  8.12062420e-01 -1.53235161e+00 ...  2.02658754e-01
  -8.23122544e-01 -3.21244628e-01]]
(6275, 20)


In [23]:
clf3 = sentiment.KNNClassifier(715) 
clf3.fit(X_pda_train, y_train)
y_pred = clf3.predict(X_pda_test) 
print(y_pred)

[1. 1. 0. ... 0. 1. 0.]


In [13]:
%%time
clf.fit(X_train.toarray(), num_iteraciones, 20)
X_pdatrain =clf.transform(X_train.toarray())



CPU times: user 140 ms, sys: 80 ms, total: 220 ms
Wall time: 222 ms


In [14]:
print(X_pdatrain.shape)

(6225, 20)
